# Clean up the small generators FERC table

**TO DO:** 
* [ ] Add fuel type for all obvious name drops (and account for outliers)
* [ ] Create ferc id column
* [ ] Reconsile fuel_type with plant_type columns
* [ ] 

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Standard libraries
import logging
import sys
import os
import pathlib
import random

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import sqlalchemy as sa
#import pickle

# Local libraries
import pudl
#from pudl.analysis.fill_ferc1_fuel_gaps import *
#from pudl.analysis.flag_ferc1_totals import *

# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 75
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [4]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq='AS'
)

ferc_engine = sa.create_engine(pudl_settings['ferc1_db'])

In [5]:
small_plants_raw = pd.read_sql("f1_gnrt_plant", ferc_engine)
small_plants = pudl_out.plants_small_ferc1()#.dropna(subset=['plant_name_ferc1'])

In [6]:
# Here we create a fake raw dfs dictionary with just the small plants df to run it through
# Zane's existing transform feature.
fake_dict = {'plants_small_ferc1': small_plants_raw}
new_dict = {}
small_plants_dict = pudl.transform.ferc1.plants_small(fake_dict, new_dict)
small_plants_out = small_plants_dict['plants_small_ferc1']

# drop rows with no plant name because we can't use that
small_plants_out = small_plants_out.dropna(subset=['plant_name_ferc1'])

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Constants

In [463]:
# If these columns are nan, we can assume it is either a header row or isn't useful
nan_cols = ['construction_year', 'net_generation_mwh', 'total_cost_of_plant', 'capex_per_mw', 'opex_total', 
            'opex_fuel', 'opex_maintenance', 'fuel_cost_per_mmbtu']

# If a potential header column has these strings, it's probably a useful header
header_strings = ['hydro', 'hyrdo', 'internal', 'wind', 'solar', 'gas', 'diesel', 'diesal', 
                  'steam', 'other', 'combustion', 'combustine', 'fuel cell', 'hydraulic', 
                  'waste', 'landfill', 'photovoltaic', 'nuclear', 'oil', 'renewable', 
                  'facilities', 'combined cycle']

# If a potential header has these strings, it is not a header...
exclude = ['#', '\*', 'pg', 'solargenix', 'solargennix', '\@', 'rockton', 'albany steam']

# ...unless it also has one of these strings
exceptions = ['hydro plants: licensed proj. no.', 'hydro license no.', 
              'hydro: license no.', 'hydro plants: licensed proj no.']

# plants with two fuel names
two_fuel_names_dict = {'las vegas solar': 'solar_pv', 'solar centaur': 'gas_turbine'}

# What we will rename the headers once we remove them as rows 
new_header_labels = {
    'hydroelectric': ['hydro', 'hyrdo'],
    'internal combustion': ['internal', 'interal', 'international combustion'],
    'combustion turbine': ['combustion turbine'],
    'combined cycle': ['combined cycle'],
    'gas turbine': ['gas'],
    'petroleum liquids': ['oil', 'diesel', 'diesal'],
    'solar': ['solar', 'photovoltaic'],
    'wind': ['wind'],
    'geothermal': ['geothermal'],
    'waste': ['waste', 'landfill'],
    'steam': ['steam'],
    'nuclear': ['nuclear'],
    'fuel_cell': ['fuel cell'],
    'other': ['other'],
    'renewables': ['renewables'],
}

# Header names that match the one's that zane used in his manual mapping (so we can 
# compare processes)
zane_header_labels = {
    'solar_pv': ['solar', 'photovoltaic'],
    'wind': ['wind'],
    'hydro': ['hydro', 'hyrdo'],
    'internal_combustion': ['internal', 'interal', 'international combustion', ],
    'combustion_turbine': ['combustion turbine', 'combustine turbine'],
    'combined_cycle': ['combined cycle'],
    'diesel_turbine': ['oil', 'diesel', 'diesal'],
    'gas_turbine': ['gas'],
    'geothermal': ['geothermal'],
    'waste_heat': ['waste', 'landfill'],
    'steam_heat': ['steam'],
    'nuclear': ['nuclear'],
    'fuel_cell': ['fuel cell'],
}

def expand_dict(dic):
    d = {}
    for k, lst in dic.items():    
        for i in range(len(lst)):
            d[lst[i]] = k
            #new_d = d.copy()
            #l.append(new_d)
    return d

import random

def get_rand_group(groups):
    random_set = random.choice(list(groups.groups.keys()))
    return groups.get_group(random_set)

## 1) Remove Bad Data

First lets remove some **obviously bad rows**. That includes: 
* Utilities that have reported NA values for all `nan_cols` for ALL PLANTS in a given year
* Rows just comprised or three or more dashes: `---`
* Rows with obvious NA plant names: `'', 'none', 'na', 'n/a', 'not applicable'`

In [377]:
# Remove utilities with all NAN rows because these won't contain anything meaningful
# spc = small_plants_clean
spc = (
    small_plants_out
    .groupby('utility_id_ferc1').filter(lambda x: ~x[nan_cols].isna().all().all())
)
# Show what was removed
print('REMOVED NAN VALUES: \n')
pd.concat([small_plants_out,spc1]).drop_duplicates(keep=False)[nan_cols].info()

REMOVED NAN VALUES: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 19552
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   construction_year    0 non-null      float64
 1   net_generation_mwh   0 non-null      float64
 2   total_cost_of_plant  0 non-null      float64
 3   capex_per_mw         0 non-null      float64
 4   opex_total           0 non-null      float64
 5   opex_fuel            0 non-null      float64
 6   opex_maintenance     0 non-null      float64
 7   fuel_cost_per_mmbtu  0 non-null      float64
dtypes: float64(8)
memory usage: 16.4 KB


In [378]:
# Remove rows with --- or '' for names
aa = spc[~spc['plant_name_ferc1'].str.contains('---')].copy()
spc1 = aa[~aa['plant_name_ferc1'].isin(['', 'none', 'na', 'n/a', 'not applicable'])].reset_index(drop=True)

# Show what was removed
print('REMOVED NAN NAMES:\n', pd.concat([spc, spc1]).drop_duplicates(keep=False).plant_name_ferc1.value_counts(), '\n')

REMOVED NAN NAMES:
                                        183
none                                    81
------------------                      25
not applicable                          22
-------------------                     16
na                                       8
n/a                                      7
-----                                    3
-----------                              3
--------------------                     2
------------------------                 1
-------------------------                1
------------                             1
----------------                         1
-------------                            1
---------------------------------        1
-----------------------------------      1
Name: plant_name_ferc1, dtype: int64 



In [380]:
# Show total rows removed
print(f'TOTAL ROWS REMOVED: {len(small_plants_out) - len(spc1)} rows. Current row total: {len(spc1)}')

TOTAL ROWS REMOVED: 590 rows. Current row total: 18969


## 2/3) Label Row Types: `header, note, total`

Instead of removing rows, now we're going to flag them as: 
* `header`: critical rows are NA and contains predefined header keywords
* `total`: critical rows are not NA and contains the word total
* `note`: critical rows are NA and there are multiple in a row for a given utility report year

We want to flag rather than get rid of these rows, because they contain useful information. For example, headers are often fuel types and notes sometimes contain ferc license ids.

Example of a note containing a FERC license:
* `spc4[spc4['row_type']=='clump']`
* `spc4[(spc4['utility_id_ferc1'] == 115) & (spc4['report_year']==1994)]`

In [381]:
# Add some new helper columns
spc2 = spc1.copy()
spc2.insert(3, 'possible_header', False)
spc2.insert(3, 'row_type', np.nan)

Notes are similar to the headers in that they don't contain any useful information in certain columns. At first, we'll lump them all together under `possible_header == True`, and then we'll tease out the ones that are note clumps vs actual headers.

In [382]:
# Label possible header rows (based on the nan cols specified above)
spc2.loc[spc2.filter(nan_cols).isna().all(1), 'possible_header'] = True

# Label good header rows (based on whether they contain key strings)
possible_header = spc2['possible_header']
good_header = spc2['plant_name_ferc1'].str.contains('|'.join(header_strings))
not_bad = ~spc2['plant_name_ferc1'].str.contains('|'.join(exclude))

spc2.loc[possible_header & good_header & not_bad, 'row_type'] = 'header'
spc2.loc[spc3['plant_name_ferc1'].isin(exceptions), 'row_type'] = 'header'

# Label total rows
spc2.loc[spc2['plant_name_ferc1'].str.contains('total'), 'row_type'] = 'total'

# What you'll see when you look at the values listed as headers and totals is
# that many of them are actually notes. These next kernals will help identify
# and label those clumps accordingly.
spc2[spc2['row_type']=='total'].plant_name_ferc1.value_counts()
spc2[spc2['row_type']=='header'].plant_name_ferc1.value_counts()

internal combustion                           293
hydro                                         230
internal combustion:                          179
hydro:                                        166
hydro plants:                                 108
                                             ... 
with e. hampton gas turbine on page 403.2.      1
other-leased:                                   1
other-steam expenses:                           1
gas turbines:                                   1
solar photovoltiac                              1
Name: plant_name_ferc1, Length: 142, dtype: int64

In [34]:
def create_groups(group, group_col):
    """Count groups of headers in a given utiltiy group.

    This function takes a utility group and regroups it by of rows where 
    possible_header = True (i.e.: all values in the specified nan_cols are NA)
    vs. False. Rows where possible_header = True can be bad data, headers, or notes. 
    The result is a DataFrame that contains one row per clump of similar adjecent
    possible_header values with columns val_col depicting the number of rows per
    possible_header clump.

    Ex: If you pass in a df with the possible_header values: True, False False, True, True,
    the header_groups output df will look like this: {'header':[True, False, True], 'val_col:
    [1, 2, 2]}.

    Args:
        group (pandas.DataFrameGroupBy): A groupby object that you'd like to condense by group_col.
        group_col (str): The name of the column you'd like to make sub groups from.

    Returns:
        pandas.DataFrame: A condensed version of that dataframe input grouped by
            breaks in fuel type over the years.

    """
    # Make groups based on consecutive sections where the group_col is alike.
    header_groups = group.groupby((group[f'{group_col}'].shift() !=
                        group[f'{group_col}']).cumsum(), as_index=False)
    
    # Identify the first (and only) group_col value for each group and count how many
    # rows are in each group.
    header_groups_df = header_groups.agg(header=(f'{group_col}', 'first'), val_count=(f'{group_col}', 'count'))

    return header_groups, header_groups_df

In [201]:
def get_header_clumps_all(df):
    """
    Remove clumps of consecutive rows flagged as possible headers.
    
    FERC has lots of note rows that are not headers but are also not useful for analysis.
    This function looks for rows flagged as possible headers (based on NAN values) and checks to
    see if there are multiple in a row. A header row is (usually) defined as a row with NAN values
    followed by rows without NAN values, so when there are more than one clumped together they are
    likely either notes or not helpful.
    
    Sometimes note clumps will end with a meaningful header. This function also checks for this and will
    unclump any headers at the bottom of clumps. There is one exception to this case which is a header that 
    is followed by a plant that had no values reported... Unfortunately I haven't built a work around,
    but hopefully there aren't very many of these. Currently, that header and plant will be categorized
    as clumps and removed.
    
    """
    util_groups = df.groupby(['utility_id_ferc1', 'report_year'])
    
    def get_header_clumps(util_year_group):
        
        # Create mini groups that count pockets of true and false for each utility and year
        # create_groups() is a function from the fill_ferc1_fuel_gaps module-- basically what
        # it does is create a df where each row represents a clump of adjecent, equal values for
        # a given column. Ex: a column of True, True, True, False, True, False, False, will
        # appear as True, False, True, False with value counts for each
        group, header_count = create_groups(util_year_group, 'possible_header')
        
        # These are used later to enable exceptions
        max_idx_val = header_count.index.max()
        max_df_val = util_year_group.index.max()
        
        # Create a list of the index values that comprise each of the header clumps
        # It's only considered a clump if it is greater than 1.
        idx_list = list(header_count[
            (header_count['header']) & (header_count['val_count'] > 1)].index)
        
        # If the last row is not a clump (i.e. there is just one value) but it is a header (i.e. has nan values)
        # then also include it in the index values to be flagged because it might be a one-liner note. And
        # because it is at the bottom there is no chance it can actually be a useful header because there are
        # no value rows below it.
        last_row = header_count.tail(1)
        if (last_row['header'].item()) & (last_row['val_count'].item()==1):
            idx_list = idx_list + list(last_row.index)
        # If there are any clumped/end headers:
        if idx_list:
            for idx in idx_list:
                # Check to see if last clump bit is not a header... sometimes you might find a clump of
                # notes FOLLOWED by a useful header. This next bit will check the last row in each of
                # the identified clumps and "unclump" it if it looks like a valid header. We only need
                # to check clumps that fall in the middle because, as previously mentioned, the last row
                # cannot contain any meaningful header information because there are no values below it.
                idx_range = group.groups[idx+1]
                is_middle_clump = group.groups[idx+1].max() < max_df_val
                is_good_header = util_year_group.loc[
                    util_year_group.index.isin(group.groups[idx+1])].tail(1)['plant_name_ferc1'].str.contains('|'.join(header_strings)).all()  #.isin(header_strings).all()
                # If the clump is in the middle and the last row looks like a header, then drop it from the idx range
                if is_middle_clump & is_good_header:
                    idx_range = [x for x in idx_range if x != idx_range.max()]
                # Label the clump as a clump
                util_year_group.loc[
                    util_year_group.index.isin(idx_range), 'row_type'] = 'note'
        return util_year_group
    
    return util_groups.apply(lambda x: get_header_clumps(x))

In [383]:
spc3 = get_header_clumps_all(spc2)

In [512]:
# Take a look at the rows that were labeled as notes. Looks like notes to me!
# We label rather than remove them because it's possible the might contain useful information
# such as ferc license number.

print('CLUMP PLANT NAMES: \n\n', spc3[spc3['row_type']=='note'].plant_name_ferc1.unique())
print('NON CLUMP OR HEADER PLANT NAMES: \n\n', spc3[~spc3['row_type'].isin(['note', 'header'])].plant_name_ferc1.unique().tolist())

CLUMP PLANT NAMES: 

 ['(a) project #2438' '(b) project #2738' '(c) project #2835'
 '(d) project #2852' 'license project: 2069'
 '(1) applicable to atlantic city' 'electrics share of jointly-'
 'owned facility.' 'note: generation provided in column (e)'
 'is in kilowatt-hours.' 'a-internal combustion unit'
 'b-combustion turbine unit' 'note: amounts per above are not'
 'included on pages 402, 403, and 403a' '(plant not yet in service)'
 'item 2. the generating unit is leased. the'
 'cost of plant represents the annual rental cost.'
 'ferc licensed project no. 2380' 'generator - hutsonville'
 '(1) operated on an emergency basis only.' 'a project # 2090'
 'b project # 2531' 'c project # 2674' 'd project # 2879'
 'cummins diesel #3' 'cummins diesel #4' 'cummins diesel #5'
 'internal combustion:' "yellowstone nat'l park:"
 '*(1) ferc licensed project no. 2582'
 '*(2) ferc licensed project no. 2584'
 '*(3) ferc licensed project no. 2596' '*(4) water for power'
 'other production' 'santa cat

In [384]:
# Now, if you take a look at the total and header rows they are a lot cleaner because we got rid of the notes!
clean_totals = spc3[spc3['row_type']=='total'].plant_name_ferc1.value_counts() # print if you want to see
clean_headers = spc3[spc3['row_type']=='header'].plant_name_ferc1.value_counts() # print if you want to see

# There are some recurring TOTAL rows that contain the plant name "(amounts are for the total of..."
# If you look here, however, only two of them contain any useful information (i.e., they're notes not totals)
num_cols = [x for x in spc3.select_dtypes(include=['float', 'Int64']).columns.tolist() if x not in ['utility_id_ferc1', 'report_year', 'ferc_license_id']]
test = spc3[spc3['plant_name_ferc1'].str.contains('amounts are for')]
print("'AMOUNTS ARE FOR'...ROWS WITH NON ZERO/NA VALUES")
display(test[test[num_cols] > 0].dropna(how='all')) # print this out if you want to see for yourself

# Lets take that one row with helpful information and copy it to the one above (where it should be)
print('\nTOTAL ROW CONTAINS INFORMATION THAT SHOULD BE IN THE ROW ABOVE')
display(spc3.iloc[8293:8295]) # the info from this total row belongs above!
spc4.loc[[8293],num_cols] = spc3.loc[8294][num_cols].values # move the information up
print('\nMOVED TOTAL VALUES TO TOTAL ROW')
display(spc3.iloc[8293:8295]) # look at this to see that both rows now have the same numeric information

# Now we can confidently call all rows with "(amounts are for the total of..." NOTE rather than TOTAL
spc3.loc[spc3['plant_name_ferc1'].str.contains('amounts are for'), 'row_type'] = 'note'

'AMOUNTS ARE FOR'...ROWS WITH NON ZERO/NA VALUES


,utility_id_ferc1,report_year,plant_name_ferc1,row_type,possible_header,construction_year,capacity_mw,peak_demand_mw,net_generation_mwh,total_cost_of_plant,capex_per_mw,opex_total,opex_fuel,opex_maintenance,fuel_type,fuel_cost_per_mmbtu,plant_name_clean,plant_type,ferc_license_id,record_id
8290,NaN,NaN,NaN,NaN,NaN,1942.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8294,NaN,NaN,NaN,NaN,NaN,2004.0,160.5,160.5,704000.0,156897178.0,977553.0,136296.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



TOTAL ROW CONTAINS INFORMATION THAT SHOULD BE IN THE ROW ABOVE


,utility_id_ferc1,report_year,plant_name_ferc1,row_type,possible_header,construction_year,capacity_mw,peak_demand_mw,net_generation_mwh,total_cost_of_plant,capex_per_mw,opex_total,opex_fuel,opex_maintenance,fuel_type,fuel_cost_per_mmbtu,plant_name_clean,plant_type,ferc_license_id,record_id
8293,210,2004,intrepid wind farm (107 units @ 1.5 mw each),NaN,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,intrepid wind farm (107 units @ 1.5 mw each),NaN,NaN,f1_gnrt_plant_2004_12_210_0_7
8294,210,2004,(amounts are for the total of all 107 units),total,False,2004.0,160.5,160.5,704000.0,156897178.0,977553.0,136296.0,NaN,NaN,,NaN,intrepid wind farm,wind,NaN,f1_gnrt_plant_2004_12_210_0_8



MOVED TOTAL VALUES TO TOTAL ROW


,utility_id_ferc1,report_year,plant_name_ferc1,row_type,possible_header,construction_year,capacity_mw,peak_demand_mw,net_generation_mwh,total_cost_of_plant,capex_per_mw,opex_total,opex_fuel,opex_maintenance,fuel_type,fuel_cost_per_mmbtu,plant_name_clean,plant_type,ferc_license_id,record_id
8293,210,2004,intrepid wind farm (107 units @ 1.5 mw each),NaN,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,intrepid wind farm (107 units @ 1.5 mw each),NaN,NaN,f1_gnrt_plant_2004_12_210_0_7
8294,210,2004,(amounts are for the total of all 107 units),total,False,2004.0,160.5,160.5,704000.0,156897178.0,977553.0,136296.0,NaN,NaN,,NaN,intrepid wind farm,wind,NaN,f1_gnrt_plant_2004_12_210_0_8


In [385]:
# There are some rows that looks like they could be headers or totals, but this shows that
# they all have information in their respective rows are are therefore correctly designated as TOTALS.
spc3[(spc3['plant_name_ferc1'].str.contains('total')) & (spc3['plant_name_ferc1'].str.contains(':'))]

,utility_id_ferc1,report_year,plant_name_ferc1,row_type,possible_header,construction_year,capacity_mw,peak_demand_mw,net_generation_mwh,total_cost_of_plant,capex_per_mw,opex_total,opex_fuel,opex_maintenance,fuel_type,fuel_cost_per_mmbtu,plant_name_clean,plant_type,ferc_license_id,record_id
13267,161,2012,total hydro:,total,False,NaN,74.3,74.0,1.983915e+08,164383888.0,61522031.0,8425540.0,NaN,3346929.0,,NaN,total hydro:,NaN,NaN,f1_gnrt_plant_2012_12_161_0_36
13285,161,2012,total solar photovoltaic:,total,False,NaN,21.0,21.0,3.214275e+07,108241238.0,68033111.0,3987417.0,NaN,NaN,,NaN,total solar photovoltaic:,solar_pv,NaN,f1_gnrt_plant_2012_12_161_1_7
16273,193,2016,total:,total,False,NaN,395.3,0.0,1.025813e+09,844115518.0,49347843.0,3830812.0,NaN,10384670.0,,NaN,total:,NaN,NaN,f1_gnrt_plant_2016_12_193_0_20
17062,193,2017,total:,total,False,NaN,398.1,0.0,1.089549e+09,839018896.0,60062594.0,4499639.0,NaN,9175033.0,,NaN,total:,NaN,NaN,f1_gnrt_plant_2017_12_193_0_21
18111,193,2018,total:,total,False,NaN,587.2,0.0,9.349530e+08,856253120.0,61444114.0,5582861.0,NaN,9102533.0,,NaN,total:,NaN,NaN,f1_gnrt_plant_2018_12_193_0_20


## 4) Apply Header Fuel Type to Relevant Rows

In [438]:
spc4 = spc3.copy()

# Clean header names
spc4['header_clean'] = np.nan
d = expand_dict(zane_header_labels)

# Map cleaned header names onto df in a new column
spc4.loc[spc4['row_type']=='header', 'header_clean'] = (
    spc4['plant_name_ferc1'].str.extract(fr"({'|'.join(d.keys())})", expand=False).map(d))

# See what wasn't cleaned up
spc4[(spc4['row_type']=='header') & (spc4['header_clean'].isna())].plant_name_ferc1.value_counts()

other                                           22
hydraulic:                                      15
other:                                           8
lewiston canal facilities:                       7
hydraulic                                        6
renewables:                                      5
other general ops. supervision & engineering     3
other production:                                2
hydraulic (1):                                   1
other-leased:                                    1
renewables                                       1
Name: plant_name_ferc1, dtype: int64

In [439]:
# Make groups based on utility, year, and header
header_groups = spc4.groupby(['utility_id_ferc1', 'report_year', (spc4['row_type']=='header').cumsum()])

# Forward fill based on headers
spc4['fuel_type'] = np.nan
spc4.loc[spc4['row_type']!='note', 'fuel_type'] = header_groups.header_clean.ffill()

In [440]:
## COME UP WITH A WAY TO CHECK FUEL TYPE OVER TIME MAYBE BY PLANT ID?

In [442]:
# 161	1994
# 29	2000 # forked up....

# Use this to see how well it worked!
util_groups = spc4.groupby(['utility_id_ferc1', 'report_year'])
get_rand_group(util_groups)[['plant_name_ferc1', 'row_type', 'header_clean', 'fuel_type', 'construction_year']]

,plant_name_ferc1,row_type,header_clean,fuel_type,construction_year
3881,steam,header,steam_heat,steam_heat,NaN
3882,warden station,NaN,NaN,steam_heat,1959.0
3883,hydro,header,hydro,hydro,NaN
3884,prickett,NaN,NaN,hydro,1931.0
3885,hoist*,NaN,NaN,hydro,1916.0
3886,mcclure,NaN,NaN,hydro,1919.0
3887,autrain,NaN,NaN,hydro,1910.0
3888,cataract,NaN,NaN,hydro,1929.0
3889,*includes silver,note,NaN,NaN,NaN
3890,lake,note,NaN,NaN,NaN


## 5) Fill-in Obvioius Fuel Types (just hydro for now)

In [497]:
spc5 = spc4.copy()

# Check for non-labeled hydro in name
non_labeled_hydro = spc5[(spc5['fuel_type']!='hydro') & (spc5['row_type']!='note') & (spc5['plant_name_ferc1'].str.contains('hydro'))].plant_name_ferc1.value_counts()

# Fill in hydro
not_note = spc5['row_type'] != 'note'
contains_hydro = spc5['plant_name_ferc1'].str.contains('hydro')
spc5.loc[not_note & contains_hydro, 'fuel_type'] = 'hydro'

In [495]:
# THERE ARE STILL LOTS OF KINDS TO WORK OUT SO JUST DOING HYDRO FOR NOW

# # label all "obvious" plant types
# fuel_dict = expand_dict(zane_header_labels)
# not_note = spc5['row_type'] != 'note'
# in_keys = spc5['plant_name_ferc1'].str.contains('|'.join(fuel_dict.keys()), regex=True)
# in_two_fuel_keys = spc5['plant_name_ferc1'].str.contains('|'.join(two_fuel_names_dict.keys()), regex=True)
# not_in_exceptions = ~spc5['plant_name_ferc1'].str.contains('windsor rd|gaston|sc-etwind')

# # replace obvious
# spc5.loc[not_note & in_keys & not_in_exceptions, 'fuel_type'] = (
#     spc5['plant_name_ferc1'].str.extract(fr"({'|'.join(fuel_dict.keys())})", expand=False).map(fuel_dict))

# # replace outliers
# spc5.loc[not_note & in_two_fuel_keys, 'fuel_type'] = (
#     spc5['plant_name_ferc1'].str.extract(fr"({'|'.join(two_fuel_names_dict.keys())})", expand=False).map(two_fuel_names_dict))

In [511]:
# Take a look
util_groups = spc5.groupby(['utility_id_ferc1', 'report_year'])
get_rand_group(util_groups)[['plant_name_ferc1', 'row_type', 'header_clean', 'fuel_type', 'construction_year']]

,plant_name_ferc1,row_type,header_clean,fuel_type,construction_year
5785,cecil lynch (internal combustion),NaN,NaN,NaN,1967.0


## 6) Add FERC ID Column

In [991]:
def add_ferc_lic_from_name(df):

    """Extract FERC license number from plant_name_ferc1 and make new column.
    
    Many of FERC license numbers are embedded in the plant_name_ferc1 field. This function
    Extracts those license numbers and puts them in a new column.
    
    """
    # Extract all numbers greater than 2 digits from plant_name_ferc1 and put then in a new column as integers.
    # Rename manually collected FERC id column to reflect that.
    df = (
        df.assign(
            ferc_license=lambda x: x.plant_name_ferc1.str.extract('(\d{3,})').astype('float').astype('Int64'),
            ferc_license_id=lambda x: x.ferc_license_id.astype('Int64'))
        .rename(columns={'ferc_license_id': 'ferc_license_manual'}))
    
    # Not all of these 3+ digit numbers are FERC licenses. Some are dates, dollar amounts, page numbers, or numbers
    # of wind turbines. These next distinctions help to weed out the non-licesnse values and keep the good ones.
    obvious_license = df.plant_name_ferc1.str.contains('no\.|license|ferc|project', regex=True)
    not_license = df.plant_name_ferc1.str.contains('page|pg|\$|wind|nonutility|units|surrendered', regex=True)
    exceptions = df.plant_name_ferc1.str.contains('tomahawk|otter rapids|wausau|alexander|hooksett|north umpqua', regex=True)
    year_vs_num = (df['ferc_license'] > 1900) & (df['ferc_license'] < 2050)
    not_hydro = ~df.plant_type.isin(['hydro', np.nan]) # figure this one out.....
    extracted_license = df.ferc_license.notna()
    
    # Replace all the non-license numbers with nan
    df.loc[extracted_license & not_hydro, 'ferc_license'] = pd.NA # figure this one out.....
    extracted_license = df.ferc_license.notna() # reset
    df.loc[extracted_license & not_license, 'ferc_license'] = pd.NA
    extracted_license = df.ferc_license.notna() #reset
    df.loc[extracted_license & year_vs_num & ~obvious_license & ~exceptions, 'ferc_license'] = pd.NA

    df['ferc_license'] = df.ferc_license.astype('Int64') # figure out how not to do this twice....
    
    return df

In [1015]:
spc6 = add_ferc_lic_from_name(spc5)

# Use these to test grabbing the license number f

#spc6[(spc6['utility_id_ferc1']==7) & (spc6['report_year']==1994)]
#spc6[(spc6['utility_id_ferc1']==17) & (spc6['report_year']==1994)]
test = spc6[(spc6['utility_id_ferc1']==151) & (spc6['report_year']==1997)].copy()
#test = spc6[(spc6['utility_id_ferc1']==57) & (spc6['report_year']==2000)]

Now we have to get the license numbers that appear in notes! These values are a few lines below the actual row they apply to.

In [1057]:
# THIS DOESN'T WORK BECAUSE DICT KEYS CANNOT CONTAIN DUPLICATES AND ALSO THERE MIGHT BE THE SAME FERC
# LICENSE OVER MULTIPLE YEARS.....MIGHT HAVE TO GROUPBY YEAR AND UTILITY
             
             
def get_license_id_from_note(group):
    
    if (group.row_type=='note').any() and (group.ferc_license.notna().any()):
        # Create dictionary of notes and license numbers
        is_note = group['row_type'] == 'note'
        ferc_id_dict = dict(zip(
            group[is_note].plant_name_ferc1.str.extract('(\(\d?[a-z]?[A-Z]?\))')[0],
            group[is_note].ferc_license))   
        print(ferc_id_dict)
        # Create the same dictionary but with regex
        regex_dict = {}
        for key, value in ferc_id_dict.items():
            regex_dict[re.escape(key)] = value

        contains_dict_key = group['plant_name_ferc1'].str.contains('|'.join(regex_dict.keys()), regex=True)

        group.loc[~is_note & contains_dict_key, 'ferc_license'] = (
            group['plant_name_ferc1'].str.extract(f"({'|'.join(regex_dict.keys())})", expand=False).map(ferc_id_dict))
    
    return group

In [1059]:
groups = spc6.groupby(['utility_id_ferc1', 'report_year'])
groups.apply(lambda x: get_license_id_from_note(x))

{nan: <NA>}
{nan: <NA>}


TypeError: decoding to str: need a bytes-like object, float found

In [1018]:
# Create dictionary of notes and license numbers
is_note = test['row_type'] == 'note'
ferc_id_dict = dict(zip(
    test[is_note].plant_name_ferc1.str.extract('(\(\d?[a-z]?[A-Z]?\))')[0],
    test[is_note].ferc_license))   

# Create the same dictionary but with regex
regex_dict = {}
for key, value in ferc_id_dict.items():
    regex_dict[re.escape(key)] = value

contains_dict_key = test['plant_name_ferc1'].str.contains('|'.join(regex_dict.keys()), regex=True)

In [1019]:
test.loc[~is_note & contains_dict_key, 'ferc_license'] = (
    test['plant_name_ferc1'].str.extract(f"({'|'.join(regex_dict.keys())})", expand=False).map(ferc_id_dict))

In [1020]:
test[['plant_name_ferc1', 'ferc_license', 'row_type', 'plant_type']]
#test[test['plant_type'].isin([np.nan])]

,plant_name_ferc1,ferc_license,row_type,plant_type
3215,rochester station #2 (1),2582,NaN,NaN
3216,rochester station #26 (2),2584,NaN,NaN
3217,mt. morris station #160 (3),2596,NaN,NaN
3218,wiscoy station #170,170,NaN,NaN
3219,mills mills station #172,172,NaN,NaN
3220,property 3 riverbed land,<NA>,NaN,NaN
3221,property 4 riverbed land,<NA>,NaN,NaN
3222,property 6 riverbed land,<NA>,NaN,NaN
3223,property 23 riverbed land,<NA>,NaN,NaN
3224,property 30 riverbed land,<NA>,NaN,NaN


In [691]:
spc6[(spc6['row_type']=='note') & (spc6['ferc_license'].notna())]

,utility_id_ferc1,report_year,plant_name_ferc1,row_type,possible_header,construction_year,capacity_mw,peak_demand_mw,net_generation_mwh,total_cost_of_plant,capex_per_mw,opex_total,opex_fuel,opex_maintenance,fuel_type,fuel_cost_per_mmbtu,plant_name_clean,plant_type,ferc_license_manual,record_id,header_clean,ferc_license
10,115,1994,(a) project #2438,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(a) project #2438,NaN,<NA>,f1_gnrt_plant_1994_12_115_0_41,NaN,2438
11,115,1994,(b) project #2738,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(b) project #2738,NaN,<NA>,f1_gnrt_plant_1994_12_115_0_42,NaN,2738
12,115,1994,(c) project #2835,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(c) project #2835,NaN,<NA>,f1_gnrt_plant_1994_12_115_0_43,NaN,2835
13,115,1994,(d) project #2852,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(d) project #2852,NaN,<NA>,f1_gnrt_plant_1994_12_115_0_44,NaN,2852
31,7,1994,license project: 2069,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,license project: 2069,NaN,<NA>,f1_gnrt_plant_1994_12_7_0_13,NaN,2069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18716,57,2019,(d) licensed project number 2341.,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(d) licensed project number 2341.,NaN,<NA>,f1_gnrt_plant_2019_12_57_0_39,NaN,2341
18717,57,2019,(e) licensed project number 2350.,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(e) licensed project number 2350.,NaN,<NA>,f1_gnrt_plant_2019_12_57_0_40,NaN,2350
18793,115,2019,(a) project #2738,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(a) project #2738,NaN,<NA>,f1_gnrt_plant_2019_12_115_0_12,NaN,2738
18794,115,2019,(b) project #2835,note,True,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(b) project #2835,NaN,<NA>,f1_gnrt_plant_2019_12_115_0_13,NaN,2835
